# Geometry

This tutorial is an introduction to how Neuropythy manages the geometry of the human cortex. 

**Author**: &nbsp;&nbsp; [Noah C. Benson](mailto:nben@uw.edu)  
**Date**: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; June 11, 2022  
**Link**: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [noahbenson/neuropythy-tutorials](https://github.com/noahbenson/neuropythy-tutorials)

## Setup

To start with, we need to import various libraries. These include, first, neuropythy itself, and, second, the ipyvolume library for 3D plotting. We also import various utility libraries like os and sys.

In [1]:
# Import some standard/utility libraries:
import os, sys, six # six provides python 2/3 compatibility

# Import our numerical/scientific libraries, scipy and numpy:
import numpy as np
import scipy as sp

# The neuropythy library is a swiss-army-knife for handling MRI data, especially
# anatomical/structural data such as that produced by FreeSurfer or the HCP.
# https://github.com/noahbenson/neuropythy
import neuropythy as ny

# Import graphics libraries:
# Matplotlib/Pyplot is our 2D graphing library:
import matplotlib as mpl
import matplotlib.pyplot as plt

# We also import ipyvolume, the 3D graphics library used by neurropythy, for 3D
# surface rendering (optional).
import ipyvolume as ipv

In [2]:
# These "magic commands" tell matplotlib that we want to plot figures inline and
# That we are using qt as a backend; due to bugs in certain versions of
# matplotlib, we put them in a separate cell from the import statements above
# and the configuration statements below.
%matplotlib inline

## Choose a Subject

We need a subject whose ROIs we are going to draw. This notebook uses the subject `'bert'`, who is included in the docker-image that is part of the tutorials repository and with any FreeSurfer installation. You can optionally configure neuropythy to know where your FreeSurfer subjects directory is, allowing you to load subjects by their names alone (see [this page](https://github.com/noahbenson/neuropythy/wiki/Configuration) for information on configuring neuropythy), but the easiest way to ensure that you load the subject you intend to is to pass the subject's full path.

In [3]:
# If you aren't running the tutorial in the docker-image, make sure to set this
# to a FreeSurfer subject directory that you have access to locally.
sub = ny.freesurfer_subject('/data/freesurfer_subjects/bert')

## Mesh Objects

Most of the geometric data of cortex is tracked by neuropythy using `Mesh` objects. These objects keep track of the various cortical surfaces (white, pial, midgray, inflated, etc.), which are represented by two pieces of data: (1) a matrix of vertex coordinates and (2) a matrix of triangle (face) indices.

We'll start by looking at the coordinates of the white-matter surface.

In [4]:
# Surfaces can be extracted by name using the surface method, or
# they can be extracted from the surfaces dictionary (though
# spherical surfaces such as the native sphere and the fsaverage
# sphere are in the registrations dictionary instead of the surfaces
# dictionary). Additionally, the white and pial surfaces can be 
# extracted using the white_surface and pial_surface aliases.
mesh = sub.lh.surface('white')
# or:
# mesh = sub.lh.surfaces['white']
# mesh = sub.lh.white_surface

In [5]:
# Coordinate matrices are always 3 x N numpy matrices (where N is the
# mesh's vertex count). If the mesh is a 2D matrix, this will be 2 x N
# instead (see the plotting-2D.ipynb tutorial for more information
# about making 2D flatmaps).
(x,y,z) = mesh.coordinates

# The shape of any of the x, y, and z coordinates should be the same
# as the vertex count of the mesh itself.
(len(x), len(y), len(z), mesh.vertex_count)

(133103, 133103, 133103, 133103)

In [6]:
# If we want to see what the point-cloud of vertices looks like, we can
# use ipyvolume to make a scatter plot of these points. This is
# typically hard to interpret, however, as point-clouds are not good
# for seeing structure.
# Note, also, that because ipyvolume, by default, doesn't force the axes
# to represent space equally, this will look like a somewhat distorted
# hemisphere.
ipv.scatter(x, y, z, size=0.5)
ipv.show()

The face matrix is the other critical part of the mesh. The faces are actually tracked by an object called a `Tesselation` that is part of both the mesh and the `Cortex` object that it came from (the `Cortex` and all of its `Mesh` surfaces share the same tesselation object).

In [7]:
sub.lh.tess

Tesselation(<266202 faces>, <133103 vertices>)

The face matrix itself is a `3 x M` integer matrix that stores vertex labels. For most meshes, the vertex labels are equivalent to `range(N)` where `N` is the number of vertices; however, for sub-meshes such as flatmaps (see the `plotting-2D` tutorial), the label for each vertex remains the same as it was for its super-mesh. This essentially means that if `flatmap = supermesh.mask_flatmap(...)` (or similar) then `flatmap.labels` is the list of vertices from `supermesh` that were included in `flatmap`.

The face matrix is stored in `tess.faces`; for flatmaps there is also `tess.indexed_faces` which uses vertex *indices* instead of vertex *labels*. So for a full surface mesh, `tess.indexed_faces` and `tess.faces` are equivalent, but for submeshes like flatmaps, `tess.indexed_faces` contains proper indices into the coordinate matrices or property vectors while `tess.faces` contains indices into the supermesh's coordinates or property vectors.

In [8]:
# An easy way to think about the face matrix is that each row corresponds to
# one of the vertices at the corner of the face.
(a,b,c) = sub.lh.tess.faces # or sub.lh.tess.indexed_faces

# These should match the face count.
(len(a), len(b), len(c), sub.lh.tess.face_count)

(266202, 266202, 266202, 266202)

The `Tesselation` object also stores the edges (and indexed edges) of the mesh. The edge matrix is similar to the face matrix, except that it is a `2 x Q` matrix where `Q` is the number of edges.

In [9]:
(u, v) = sub.lh.tess.edges # or sub.lh.tess.indexed_edges
(len(u), len(v), sub.lh.tess.edge_count)

(399303, 399303, 399303)

In [10]:
nei = sub.lh.tess.neighborhoods

(len(nei), sub.lh.vertex_count)

(133103, 133103)

In [11]:
# Pick a vertex and see what its neighbors are.
vid = 100
nei[100]

(96, 202, 210, 209, 99)

In [12]:
# Verify that these are the correct vertices using the edges or faces.
(u,v) = sub.lh.tess.edges
ii = (u == vid)
jj = (v == vid)
v_neis = np.concatenate([v[ii], u[jj]])
np.unique(v_neis)

array([ 96,  99, 202, 209, 210])